## Control Variate Sampling 


In [1]:
import numpy as np

In [2]:
def f(x):
    value = 1 /(1+x)
    return value

In [3]:
def g(x):
    value = 1 + x
    return value

In [4]:
truth = 3.0 / 2.0

In [5]:
truth

1.5

### The Naive Solution

In [6]:
n = 1500
u = np.random.uniform(size=n)
x1 = f(u)
x1.mean()
x1.var()
x1.std()
se = x1.std() / np.sqrt(n)

In [7]:
x1.mean()

0.6940299592889958

In [8]:
se

0.003638341045535219

### The Control Variate Solution

In [9]:
c = 0.4773
y = g(u)
x2 = f(u) + c * (g(u) - truth)

In [10]:
x2.mean()

0.6934802689511264

In [11]:
se2 = x2.std() / np.sqrt(n)

In [12]:
se2

0.0006391291218281802

## Naive Monte Carlo in a BS World

In [13]:
import time
t1 = time.time()

In [14]:
def VanillaCallPayoff(spot, strike):
    return np.maximum(spot - strike, 0.0)

In [30]:
# The same old same old parameters

S = 41.0
K = 40.0
r = 0.08
v = 0.30
q = 0.0
T = 1.0
M = 10000 # number of MC replications
N = 252   # number of MC steps in a particular path

In [31]:
dt = T
nudt = (r - q - 0.5 * v * v) * dt
sigdt = v * np.sqrt(dt)

In [32]:
spot_t = np.empty((N))
call_t = np.empty(M)

z = np.random.normal(size=(M,N))

for i in range(M):
    spot_t[0] = S
    for j in range(1,N):
        spot_t[j] = spot_t[j-1] * np.exp(nudt + sigdt * z[i,j])
    call_t[i] = VanillaCallPayoff(spot_t[-1], K)

In [33]:
call_prc = np.exp(-r * T) * call_t.mean()
t2 = time.time()

In [34]:
call_t.mean()

15940367262.238596

In [35]:
se = call_t.std() / np.sqrt(M)

In [36]:
se

9924321587.235249

In [37]:
print("The Naive Monte Carlo Price is: {0:.3f}".format(call_prc))
print("The Naive Monte Carlo StdErr is: {0:.6f}".format(se))
print("The total time take: {0}".format(t2-t1))

The Naive Monte Carlo Price is: 14714813587.179
The Naive Monte Carlo StdErr is: 9924321587.235249
The total time take: 179.82880783081055


### The Control Variate Approach in a BS World

We will use the BS-Delta formula for our control variate. We can write the BS Delta function as follows:

In [38]:
from scipy.stats import norm

In [39]:
def BlackScholesDelta(spot, t, strike, expiry, volatility, rate, dividend):
    tau = expiry - t
    d1 = (np.log(spot/strike) + (rate - dividend + 0.5 * volatility * volatility) * tau) / (volatility * np.sqrt(tau))
    delta = np.exp(-dividend * tau) * norm.cdf(d1) 
    return delta

In [40]:
erddt = np.exp((r - q) * dt)    
beta = -1.0

spot_t = np.empty((N))
call_t = np.empty(M)
#cash_flow_t = np.zeros((engine.replications, ))
z = np.random.normal(size=(M,N))

for i in range(M):
        #spot_t = spot
        convar = 0.0
        #z = np.random.normal(size=int(engine.time_steps))
        spot_t[0] = S
        
        for j in range(1,N):
            t = j * dt
            delta = BlackScholesDelta(S, t, K, T, v, r, q)
            spot_t[j] = spot_t[j-1] * np.exp(nudt + sigdt * z[i,j])
            convar += delta * (spot_t[j] -  spot_t[j-1]* erddt)
            #spot_t = spot_tn

        call_t[i] = VanillaCallPayoff(spot_t[-1], K) + beta * convar

/home/brough/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in double_scalars
  This is separate from the ipykernel package so we can avoid doing imports until
/home/brough/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in sqrt
  This is separate from the ipykernel package so we can avoid doing imports until


In [41]:
disc = np.exp(-r * T)
call_prc = disc * call_t.mean()

In [42]:
nudt

0.035

In [43]:
spot_t = np.empty((N))
call_t = np.empty(M)

z = np.random.normal(size=(M,N))

for i in range(M):
    spot_t[0] = S
    for j in range(1,N):
        spot_t[j] = S * np.exp(nudt + sigdt * z[i,j])
    call_t[i] = VanillaCallPayoff(spot_t[-1], K)

In [44]:
call_t.mean()

7.574918158543756